<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Равновесное моделирование методом Монте - Карло двумерной модели Изинга

## Examples - Statistical Mechanics
<section class="post-meta">
By Niels Henrik Aase, Asle Sudbø, Eilif Sommer Øyre, and Jon Andreas Støvneng
</section>
Last edited: September 29nd 2019

___

## Вступление

Модель Изинга, пожалуй, является наиболее тщательно исследованной моделью в статистической физике. Основными причинами этого являются ее простота и тот факт, что она имеет аналитическое решение на двумерной бесконечной квадратной решетке, что делает ее отличным эталоном для схем численных расчетов. Это решение [[1]](#rsc) было разработано Нобелевским лауреатом Ларсом Онсагером в 1944 году, бывшим студентом NTNU, что делает проект особенным для нас в NumFys. Модель Изинга является единственным нетривиальным примером фазового перехода, который может быть строго доказан [[2]](#rsc). Однако доказательство выходит далеко за рамки этого проекта.

Любопытный читателя вывод удельной теплоты можно найти в статье Ларса Онсагера [[1]](#rsc). Выражение для спонтанной намагниченности также было найдено Онсагером, но вывод этого выражения никогда не публиковался. Онсагер просто заявил об этом на конференции в Корнельском университете в 1948 году, а четыре года спустя К. Н.Янг опубликовал вывод того же результата [[3]](#rsc).

В этом блокноте мы будем использовать метод Монте-Карло, возможно, самый важный вычислительный статистический алгоритм во всей физике. Это хорошо согласуется с лежащей в основе физической системой, поскольку модель Изинга была одной из самых важных моделей в статистической физике в прошлом веке. Таким образом, это тема, в которой вы можете узнать много важных принципов, касающихся алгоритмов и вычислений, которые все еще очень актуальны сегодня!

## Теория

### Модель Изинга

Модель Изинга уже обсуждалась в предыдущем [блокноте](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ising_model.ipynb), но некоторые результаты стоит повторить. Модель Изинга представляет магнитные дипольные моменты атомных спинов. Спины могут находиться в двух состояниях: +1 или -1. Спины взаимодействуют только со своим ближайшим соседом через параметр взаимодействия $J_{ij}$.

Рассмотрим решетку $ N \times N $, где $s_{i}$ обозначает спин на узле $i$. Мы допускаем различные взаимодействия вдоль столбцов и строк решетки, где мы обозначаем взаимодействие между спинами вдоль строк как $J$ и между спинами вдоль столбцов $J'$. Без потери общности мы предполагаем, что магнитное поле $\vec{B}$ имеет вид $\vec{B} = (0 ,0 , B)$ и что дипольные моменты имеют вид $s_i = (0, 0, \pm 1)$. Тогда гамильтониан системы становится 

\begin{equation}
H = - \sum_{<i,j>} J_{ij} s_i s_j - B\sum_{i} s_i,
\label{Hamiltonian} \quad(1)
\end{equation}
где $B$ - магнитное поле. Первое суммирование выполняется по ближайшим соседям каждой точки решетки, в то время как второе суммирование выполняется по всем точкам решетки $N^2$. Взаимодействие между ближайшими соседями описывается с помощью $J$ и $J'$.

Затем задается статфункция
\begin{equation}
Z = \sum_{\{s_i\}}  e^{-\beta H},
\label{Partition} \quad(2)
\end{equation}
где суммирование происходит по возможным состояниям системы, а $\beta$ - множитель Лагранжа, равный $\frac{1}{k_B T}$. $k_B$ - постоянная Больцмана, а $T$ - температура. С этого момента мы устанавливаем $k_B = 1$, так что $\beta$ - это просто обратная температура. Мы также предпочитаем в основном игнорировать единицы вычисляемых величин.

Число возможных состояний увеличивается непостижимо быстро, когда мы увеличиваем размер решетки. Суммирование содержит $2^{N^2}$ членов, поэтому для $N=6$ нам нужно суммировать более 68 719 476 736 спиновых конфигураций. В предыдущем [блокноте модели Изинга](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ising_model.ipynb), сумма была рассчитана точно, и, используя высокоскоростной язык, такой как Fortran, можно смоделировать решетки $5 \times 5$ за разумное время. Однако решение Онсагера справедливо только для решетки $N \times \infty$, поэтому нам нужен другой метод для моделирования более крупных систем, которые представляют больший интерес. В этом блокноте мы будем использовать метод Монте-Карло, а более конкретно алгоритм Метрополиса.


### Алгоритм Метрополиса

Алгоритм Метрополиса - это, пожалуй, самый известный алгоритм Монте-Карло. В нашем случае мы будем использовать его для аппроксимации математических ожиданий системы, т.е. макроскопических и измеримых значений, таких как удельная теплоемкость, $c$ или намагниченность, $m$. Мы разделим объяснение алгоритма на две части, сначала мы сосредоточимся на реализации одной развертки Монте-Карло, а затем объясним, как следует проводить измерения системы. Последняя, пожалуй, самая тонкая и продвинутая часть блокнота.

#### Реализация

1. Инициализация конфигурации вращения. В нашем случае начальная конфигурация спина будет случайной, соответствующей высокотемпературной начальной конфигурации.
2. Вычисление энергии конфигурации, используя (1)
3. Выбор случайного участка $i$ на решетке и вычисление энергии конфигурации, если $s_i \rightarrow -s_i$. Затем вычисление изменения энергии всей конфигурации, $\Delta E$.
4. Если $\Delta E$ отрицательна, принять изменение конфигурации спина, разрешив $s_i \rightarrow -s_i$.
5. Если $\Delta E$ неотрицательна, сгенерировать случайное число $r \in \big \langle 0, 1 \big \rangle$. Если $e^{-\beta \Delta E} > r$, изменить конфигурацию вращения, разрешив $s_i \rightarrow -s_i$.
6. Повторить процесс $N^2$ раз. Это определяет одну развертку Монте-Карло, $t$.

Подводя итог, можно сказать, что *будет* принято действие, которое уменьшает энергию конфигурации спина, в то время как шаг, который увеличивает энергию, *может будет* принят.

Важно отметить, что случайный, недетерминированный характер моделирования по методу Монте-Карло означает, что, хотя две модели имеют одинаковую начальную конфигурацию спина и внешние параметры, две модели будут развиваться по-разному, поэтому не волнуйтесь, если ваши результаты будут отличаться от моделирования к моделированию.

Несмотря на то, что мы используем стохастическое суммирование, этот алгоритм будет требователен к вычислениям для более крупной системы, его сложность масштабируется как $N^2$. Общее время выполнения программы будет зависеть от выполнения огромного количества разверток Монте-Карло, в нашем случае в диапазоне до 600 000 разверток для $N=256$, что соответствует выполнению шагов 1-5 примерно $10^{11}$ раз! Если мы дополнительно моделируем для разных температур, мы запускаем самые основные части нашего алгоритма триллионы раз. Для этого требуется чрезвычайно эффективная функция развертки Монте-Карло. К счастью, есть некоторые вещи, которые мы можем сделать, чтобы улучшить время выполнения.

Мы в основном занимаемся вычислением средней намагниченности на спин системы, $<m>$. 

$$ 
<m> = \frac{1}{N^2} \sum_{i}<s_i>, 
$$
и удельная теплоемкость при постоянном поле $C_B$ на спин. Через классическую статистическую физику и термодинамику можно показать, что $C_B$ равна

$$
\frac{C_B}{k_B} = \big[ \langle (\beta H )^2 \rangle - \langle (\beta H) \rangle ^2 \big] = \mathrm{Var}[\beta H]. 
$$

Этот фантастический и удивительный результат показывает, что удельная теплоемкость полностью определяется флуктуациями энергии (т.е. дисперсией энергии системы)! Важно отличать математическое ожидание (обозначаемое $<m>$) намагниченности от фактической намагниченности данной конфигурации спина, $m$. К сожалению, эти два слова часто используются взаимозаменяемо. Это связано с тем, что средняя намагниченность - это макроскопическая величина, которую мы измеряем с помощью экспериментов, и хотя мы строго говорим о средней намагниченности $<m>$, мы часто просто обозначаем ее $m$. Это обозначение мы будем использовать для остальной части блокнота.

Начнем с простейшего случая-средней намагниченности на спин $m$. Чтобы вычислить это, мы вычисляем намагниченность решетки после каждой развертки, а затем вычисляем среднее значение. Однако для этого требуется, чтобы сумма $\sum_{i} s_i$ была рассчитана $N^2$ раз за развертку, что является слишком трудоемкой операцией. Эту операцию можно значительно упростить, отметив, что изменение $m$ зависит только от изменения спина на случайном сайте $k$, $s_k \rightarrow-s_k$. Обозначая новое и старое состояние системы как $\nu$ и $\mu$, мы наблюдаем, что

\begin{equation}
\Delta m = m_{\nu} - m_{\mu} = \sum_{i} s_i^{\nu} - \sum_{i} s_i^{\mu} = s_k^{\nu} - s_k^{\mu} = 2 s_k^{\nu}, 
\label{Delta_m}
\end{equation}

где $s_k^{\nu}$ и $s_k^{\mu}$ обозначают вращение на сайте $k$ после и до изменения вращения. Все остальные члены в двух суммациях отменяются (потому что $s_k$ - это единственный спин, который изменяется, когда система переходит из состояния $\mu$ в состояние $\nu$). Таким образом, нам нужно вычислить $m$ только один раз, а затем каждый раз, когда спин переворачивается, т.е. $s_k \rightarrow-s_k$, мы находим обновленную намагниченность по 

$$
m_{\nu} = m_{\mu} + 2 s_k^{\nu}.
$$

Изменение энергии немного сложнее, но применяется тот же принцип: каждый раз, когда меняется спин, это влияет только на конфигурацию спина локально, нам не нужно пересматривать всю решетку. Из уравнения (1) мы можем вычислить изменение энергии

$$
\Delta E = E_{\nu} - E_{\mu} = - \sum_{<i,j>} J_{ij} s_i^{\nu} s_j^{\nu} - B\sum_{i} s_i^{\nu} + \sum_{<i,j>} J_{ij} s_i^{\mu} s_j^{\mu} + B\sum_{i} s_i^{\mu} = - \big( \sum_{i \, \mathrm{ n.n\, to } \, k} J_{ik} s_i^{\mu} (s_k^{\nu} - s_k^{\mu}) \big) - 2 B s_k^{\nu}, 
$$
где мы использовали уравнение (2) для оценки суммирования с участием $B$. Взаимодействия с ближайшими соседями более сложны, но мы наблюдаем, что только ближайшие соседи сайта $k$ релевантны для вычисления $\Delta E$, все остальные взаимодействия отменяются. Выражение можно еще больше сократить, отметив, что $(s_k^{\nu} - s_k^{\mu}) = - 2 s_k^{\mu}$ и что $s_k^{\mu}$ можно вытащить за пределы суммы. Окончательное выражение для $\Delta E$ может быть выражено следующим образом 

\begin{equation}
\Delta E = s_k^{\mu} \big( \sum_{i \, \mathrm{ n.n\, to } \, k} J_{ik} s_i^{\mu}\big) - 2 B s_k^{\nu}.
\label{Delta_E}
\end{equation}

Упрощения допустимы в любом количестве измерений, где количество измерений определяет количество членов в суммировании. В нашем случае, когда мы рассматриваем двумерную систему, суммирование содержит четыре члена.

Вы можете заметить, что почти все вычисления включают только целые числа, единственными исключениями являются случайное число $r$ и экспоненты. Для данной комбинации $J$, $J'$ и $B$ существует конечное число возможных значений $\Delta E$, таким образом, можно сохранить возможные экспоненциальные значения $e^{-\beta \Delta E}$ в начале моделирования, поэтому для любого $\Delta E$ мы знаем $e^{-\beta \Delta E}$ без необходимости вычислять его! Это не делается в нашем коде, но это еще больше улучшит время выполнения, потому что все вычисления, за исключением $r$, будут включать только целые числа, которые обычно намного быстрее, чем вычисления с действительными числами.  

Как упоминалось ранее, почти все время выполнения будет потрачено на запуск этого алгоритма до 600 000 раз. Алгоритм имеет временную сложность , которая масштабируется как $O(N2)$, поэтому код должен быть чрезвычайно эффективным. Именно поэтому мы решили запустить эту часть кода через Fortran, который обеспечивает превосходную вычислительную скорость, улучшая время выполнения в 200 раз. Для полноты картины мы также покажем, как можно было бы реализовать алгоритм на Python, так как легче понять, как алгоритм работает на языке высокого уровня, таком как Python.


## Реализация на Python

Как мы увидим позже, существует только аналитическое решение для термодинамических величин на бесконечной решетке. Решение Ларса Онсагера справедливо только для $B = 0$, а аналитическое решение для $B \neq 0$ еще не найдено. По этой причине мы решили установить $B=0$, а также установить $J = J' = 1$.

In [ ]:
# Imporing necessary packages
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import scipy.special
%matplotlib inline

newparams = {'figure.figsize': (15, 7), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

In [ ]:
def Hamiltonian(S, B= 0, J_v= 1, J_p=1):
    """Given a lattice configuration (i.e. a spin matrix) S, and the parameters B, J_v and J_p, this functions
     returns the energy of the lattice.
        
        Parameters:
            S: Spin matrix, (NxN) array
            B, J_v, J_p: Interactation and external parameters
        Returns:
            E: Energy of the lattice
    """
    B_contribution = B * np.sum(S) # Energy of the spin configuration resulting from an external magnetic field
    
    # Using np.roll here automatically satisifies the periodic boundary conditions (PBCs).
    J_v_contribution = J_v * np.sum(S * np.roll(S, 1, axis= 0)) # Energy resulting from lattice interactions along columns
    J_p_contribution = J_p * np.sum(S * np.roll(S, 1, axis= 1)) # Energy resulting from lattice interactions along rows
    E = - (J_v_contribution + J_p_contribution + B_contribution)
    return E


def delta_Hamiltonian(S, i, k, N, B = 0, J_v = 1, J_p = 1):
    """Calculates the energy change of the lattice if lattice site S[i][k] changes it's spin, in accordance
    to the simplifications in the theory part.
        
        Parameters:
            S: Spin matrix, (NxN) array
            i, k: indicates the lattice site
            N: Number of columns and rows in the lattice
            B, J_v, J_p: Interactation and external parameters
        Returns:
            delta_E: The energy change
        """
    # Here we use the modulo operator to accomodate the PBCs 
    nn = J_p * (S[(i + 1) % N][k] + S[i - 1][k]) + J_v * (S[i][(k + 1) % N] + S[i][k - 1])
    delta_E = 2 * S[i][k] * (nn + B)
    return delta_E


def Attempt_flip(S, beta, i, k, N, r, B = 0, J_v = 1, J_p = 1):
    """The central part of the Metropolis algorithm. Calculates the energy change at a random lattice site, and
    updates the spin of the lattice site in accordance to the rules of the Metropolis algorithm. Returns
    True/False on whether the spin should flip as well as the change in energy.
    
        Parameters:
            S: Spin matrix, (NxN) array
            beta: Inverse temperature
            i, k: indicates the lattice site
            N: Number of columns and rows in the lattice
            r: Random number between 0 and 1
            B, J_v, J_p: Interactation and external parameters
        Returns:
            delta_E: The energy change
            flip : Boolean value on wheter the spin should be flipped.
        """
    delta_E = delta_Hamiltonian(S, i, k, N, B, J_v, J_p)
    if delta_E <= 0:
        flip = True # If the energy change is negative, the spin flips
        return flip, delta_E
    else:
        flip = np.exp(-beta * delta_E) > r # The flip might flip, depending on r and the temperature
        return flip, delta_E



def sweep(S, beta, N, B = 0, J_v = 1, J_p = 1):
    """A full Monte Carlo sweep. Runs the previous functions N ** 2 times, while always storing the updated change in
    magnetization and energy. Returns the updated spin configuaration and the change in magnetization and
    energy (with respect to the initial values).
    
        Parameters:
            S: Spin matrix, (NxN) array
            beta: Inverse temperature
            N: Number of columns and rows in the lattice
            B, J_v, J_p: Interactation and external parameters
        Returns:
            S: Updated spin configuartion after one Monte Carlo sweep
            delta_He_sweep: Change in energy after one Monte Carlo sweep
            delta_m_sweep: Change in magnetization after one Monte Carlo sweep
        """
    delta_m_sweep = 0
    delta_He_sweep = 0
    # Generating random numbers to genereate random lattice sites 
    rand_list_index_1 = np.random.randint(N, size=N ** 2)
    rand_list_index_2 = np.random.randint(N, size=N ** 2)
    # Generating N ** 2 random numbers between 0 and 1
    rand_numbers = np.random.rand(N**2)
    for l in range(N ** 2):
        i = rand_list_index_1[l]
        k = rand_list_index_2[l]
        check, delta_E = Attempt_flip(S, beta, i, k, N, rand_numbers[l], B, J_v, J_p)
        if check:
            delta_m = (-2) * S[i][k] / (N ** 2)
            S[i][k] *= (-1)
        else:
            delta_m = 0
            delta_E = 0
        delta_He_sweep += delta_E
        delta_m_sweep += delta_m
    return S, delta_He_sweep, delta_m_sweep



def compute_quantities(S, beta, M, B = 0, J_v = 1, J_p = 1, show_time= False):
    """Given the number of wanted sweeps, M, the simulation is ran for M Monte Carlo sweeps. Returns arrays with
    the values of magnetization and energy after each sweep.
    
        Parameters:
            S: Spin matrix, (NxN) array
            beta: Inverse temperature
            M: Number of Monte Carlo sweeps
            B, J_v, J_p: Interactation and external parameters
        Returns:
            He_list: The energy after each sweep (Mx1) array
            m_list : The magnetization after each sweep (Mx1) array
        """
    start = time.time()
    N = np.shape(S)[0]
    He_list = np.zeros(M)
    He_list[0] = Hamiltonian(S, B, J_v, J_p) # Initial energy of the lattice
    m_list = np.zeros(M)
    m_list[0] = np.sum(S) / N ** 2 # Iniitial magnetization of the lattice
    for j in range(1, M):
        S, delta_He_sweep, delta_m_sweep = sweep(S, beta, N, B, J_v, J_p)
        He_list[j] = He_list[j-1] + delta_He_sweep # Storing the updated energy
        m_list[j] = m_list[j-1] + delta_m_sweep # Storing the updated magnetization
    if show_time:
        print("Iteration time with T = %.2f: %.2f" %(1 / beta, time.time() - start))
    return He_list, m_list

In [ ]:
N = 128
S = 2 * np.random.randint(2, size=(N, N))-1 # Quick way to generate a random initial spin configuration
He_list, m_list = compute_quantities(S, 1, 1000, show_time= True)

step_list = np.arange(0, 1000)

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(step_list, He_list)
plt.title(r"Energy as a function of Monte Carlo sweeps with $N$ = {}".format(N) + " and $T$ = {}".format(T) + " K")
plt.xlabel(r"$t$")
plt.ylabel(r"$E$")
plt.show()

plt.subplot(2, 1, 2)
plt.plot(step_list, m_list)
plt.title(r"Magnetization as a function of Monte Carlo sweeps with $N$ = {}".format(N) + " and $T$ = {}".format(T) + " K")
plt.xlabel(r"$t$")
plt.ylabel(r"$m$")
plt.show()

## Реализация Fortran

Как мы можем видеть выше, выполнение 1000 разверток на решетке $128 \times 128$ занимает довольно много времени. Кроме того, намагниченность не стабилизируется в течение первых 1000 разверток, поэтому ясно, что нам нужно выполнить больше разверток. Существует дальнейшая оптимизация, которую можно выполнить с помощью кода, но мы просто реализуем ту же функциональность в Fortran, чтобы получить достаточную скорость вычислений.

Код Фортрана можно найти в приложении. Для получения дополнительной информации о том, как вызывать скрипты Fortran из Python, ознакомьтесь с нашим руководством [здесь](https://www.numfys.net/howto/F2PY/).

In [ ]:
# Testing that imported fortran code works
import ising_Monte
print(ising_Monte.__doc__)

In [ ]:
# random_sweep from Fortran replaces the former sweep function written in Python

def compute_quantities_fortran(S, beta, M, B = 0, J_v = 1, J_p = 1, show_time = False):
    """Given the number of wanted sweeps, M, the simulation is ran for M Monte Carlo sweeps. Returns arrays with
    the values of magnetization and energy after each sweep. Calls the function random_sweep from Fortran.
    
        Parameters:
            S: Spin matrix, (NxN) array
            beta: Inverse temperature
            M: Number of Monte Carlo sweeps
            B, J_v, J_p: Interactation and external parameters
        Returns:
            He_list: The energy after each sweep (Mx1) array
            m_list : The magnetization after each sweep (Mx1) array
        """
    start = time.time()
    N = np.shape(S)[0]
    He_list = np.zeros(M)
    He_list[0] = Hamiltonian(S, B, J_v, J_p)
    m_list = np.zeros(M)
    m_list[0] = np.sum(S) / N ** 2
    S_f = np.copy(S)
    for j in range(1, M):
        rand_list_index1 = np.random.randint(1, N + 1, size= N ** 2) # Fortran is 1-indexed
        rand_list_index2 = np.random.randint(1, N + 1, size= N ** 2)
        rand_numbers = np.random.rand(N ** 2)
        del_He_sweep, del_m_sweep, S_f = ising_Monte.random_sweep(S_f, beta, B, J_v, J_p, rand_numbers, rand_list_index1, rand_list_index2, N, N ** 2)
        He_list[j] = He_list[j-1] + del_He_sweep
        m_list[j] = m_list[j-1] + del_m_sweep
    if show_time:
        print("Iteration time with T = %.2f: %.2f" %(1 / beta, time.time() - start), ", \t # iterations: ", M, ", \t N = ", N)
    return He_list, m_list


In [ ]:
N = 128
S = 2 * np.random.randint(2, size=(N, N))-1
T = 1
He_list, m_list = compute_quantities_fortran(S, 1 / T, 1000, show_time=True)

step_list = np.arange(0, 1000)

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(step_list, He_list)
plt.title(r"Energy as a function of Monte Carlo sweeps with $N$ = {}".format(N) + " and $T$ = {}".format(T) + " K")
plt.xlabel(r"$t$")
plt.ylabel(r"$E$")
plt.show()

plt.subplot(2, 1, 2)
plt.plot(step_list, m_list)
plt.title(r"Magnetization as a function of Monte Carlo sweeps with $N$ = {}".format(N) + " and $T$ = {}".format(T) + " K")
plt.xlabel(r"$t$")
plt.ylabel(r"$m$")
plt.show()

Время выполнения моделирования примерно в 200 раз быстрее, чем с Python, поэтому можно с уверенностью сказать, что наша функция Fortran является важным инструментом для выполнения больших симуляций.

# Измерения

#### Время корреляции

Прежде чем мы начнем вычислять интересные величины, нам нужно определить, как и когда мы должны проводить измерения системы. Нас интересуют математические ожидания $ <m>, <E> и <E^2>$, и для нахождения этих значений мы используем среднее значение в качестве оценки. Среднее значение является объективной оценкой, но для получения надежных результатов измерения должны быть независимыми. Если бы мы измерили намагниченность, а затем снова измерили ее только через одну развертку Монте-Карло, то ясно, что две конфигурации спинов будут коррелировать, значительная часть спинов не изменится. Поэтому нам нужно убедиться, что мы достаточно долго ждем между измерениями, чтобы убедиться, что конфигурация спина значительно отличается от состояния при первом измерении. Под значительными различиями мы подразумеваем, что количество спинов, которые являются такими же, как и в исходном состоянии, не превышает того, что вы ожидаете найти случайно [[4]](#rsc).

Количество разверток Монте-Карло, которые занимает этот процесс, определяется как время корреляции $\tau$. Это связано с корреляцией следующим образом. Пусть $\chi(t)$ - автокорреляционная функция, она имеет максимальное значение 1, полученное при $t=0$, где измерения полностью коррелированы. $t$ представляет количество разверток Монте-Карло. Обратное было бы, если бы система была точно противоположна исходной конфигурации, делая $\chi(t) = -1$. По мере увеличения $t$ корреляция будет уменьшаться по мере того, как текущее состояние (конфигурация спина) будет все меньше и меньше коррелировать с исходным состоянием. $\tau$ связано с $\chi(t)$ как 

\begin{equation}
\chi(t) \sim e^{- \frac{t}{\tau}}.
\label{corr_time} \quad(2)
\end{equation}

Таким образом, $\tau$ является параметром подгонки, который мы можем использовать библиотеку scipy для определения оптимальной подгонки. Наиболее естественным определением статистических независимых измерений являются измерения, выполняемые каждые $2\tau$ [[4]](#rsc). Если мы запустили $n$ развертки Монте-Карло, у нас будет $m = \frac{n}{2\tau}$ независимых измерений. Подходящая оценка для $\chi$ для измеряемой величины $O$ задается 

\begin{equation}
\chi_O(t) = \frac{1}{\chi_0} \frac{1}{n - t} \sum_{t' = 0}^{n - 1 -t} (O(t') - < O >_0 ) (O(t' + t) - < O >_t),
\label{corr_func} \quad(3)
\end{equation}

где $O(0), O(1), ..., O(n-1)$ - это $n$ измерений $O$, а $\chi_0$ определяется таким образом, что $\chi(0) = 1$. $< O >_t)$ и $< O >_0)$ определяются как

\begin{equation}
< O >_0 = \frac{1}{n - t} \sum_{t' = 0}^{n - 1 -t} O(t') \quad \quad < O >_t = \frac{1}{n - t} \sum_{t' = 0}^{n - 1 -t} O(t' + t).
\label{help_sizes}
\end{equation}

Комбинируя уравнения (2) и (3), т. е. используя нашу оценку времени корреляции $\chi$ и находя оптимальное соответствие для $\tau$, мы можем сделать хорошую оценку для $\tau$. В нашем случае мы включаем только те измерения, которые имеют положительную автокорреляцию, такую что $\chi_O(t) > 0$.
Следует отметить, что даже если мы измеряем физические величины только каждые $2\tau$, нам все равно нужно сделать много измерений, чтобы наша оценка была близка к истинному значению.


Наконец, мы отмечаем, что при экстремально низких температурах почти не будет изменений в конфигурации спина, поскольку решетка получила самую низкую возможную энергию, и крайне маловероятно, что какие-либо спины перевернутся из-за тепловых возбуждений. В этом случае мы не вычисляем время корреляции, а просто используем каждое десятое измерение.


In [ ]:
def auto_corr_func(t, tau):
    """Dummy function declariation that is needed to use the curve_fit function from the scipy library. 
    
        Parameters:
            t: The enumuration of the Monte Carlo sweeps 
            tau: Correlation time
        Returns:
            chi: exp(-t / tau)
        """
    chi = np.exp(-t / tau)
    return chi


def correlation_time(arr):
    """Calculates the estimator for the autocorrelation function, chi, as a function of t. Stops the calculations
    of chi when chi(t) becomes egative. Then calculates the correlation time by using the curve_fit
    function from scipy-library.
    
        Parameters:
            arr: The measurement series we want to determine the correlation time of
        Returns:
            tau: The correlation time
            chi: The autocorrelation function as a function of # Monte Carlo sweeps. The size of the array varies.
        """
    chi = np.zeros(0)
    n = np.minimum(len(arr), 10000)
    # Here we take the minimum of the length of the array, and 10000 to obtain a correlation time that is
    # independant of the length of the array
    for t in range(n):
        frac = (1 / (n - t))
        
        # plus 1 to include last term
        expectation_0 = frac * np.sum(arr[:(n - 1 - t) + 1])
        expectation_t = frac * np.sum(np.roll(arr, - t)[:(n - 1 - t) + 1])
        
        temp = frac * np.sum((arr[:n - 1 - t + 1] - expectation_0) *
                             (np.roll(arr, - t)[:n - 1 - t + 1] - expectation_t))
        if temp < 0:
            break
        chi = np.append(chi, temp)
    # Normalize the autocorrelation function
    chi = chi / chi[0]
    t = np.arange(0, len(chi))
    # Calculates the correlation time with scipy
    tau = scipy.optimize.curve_fit(auto_corr_func, t, chi)[0]
    return tau[0], chi


Поучительно посмотреть, как $\tau$ изменяется в зависимости от температуры. Мы также увидим, насколько хорошо работает подгонка кривой. В этом случае измеренная величина $O$ соответствует $m$, но расчеты будут выполнены так же, как и при использовании $E$.

In [ ]:
N = 128
S = 2 * np.random.randint(2, size=(N, N)) - 1
T = 2.1
m_list = compute_quantities_fortran(S, 1/T, 80000)[1]

tau, chi = correlation_time(m_list[70000:])
t = np.arange(len(chi))
print("Tau is equal to " + "%.1f" % tau + " when T = ", T, " K" )
plt.plot(t, np.exp(- t / tau), label= r"$\chi(t) \sim e^{- \frac{t}{\tau}}$")
plt.plot(t, chi, label = "Estimator for $\chi(t)$")
plt.legend()
plt.title(r"Different estimators for the autocorrelation function $\chi(t)$")
plt.xlabel(r"$t$")
plt.ylabel(r"$\chi(t)$")
plt.show()

T_list = np.linspace(1.5, 4, 20)
tau_list = np.zeros(0)
for T in T_list:
    S = 2 * np.random.randint(2, size=(N, N)) - 1
    beta = 1 / T
    # Why we differentiate cold and hot simulations is explained later
    if T < 2.5:
        m_list = compute_quantities_fortran(S, beta, 200000)[1]
        # The reason we only use the last 20000 values of the list is explained later
        tau, chi = correlation_time(m_list[180000:])

        tau_list = np.append(tau_list, tau)
    else:
        m_list = compute_quantities_fortran(S, beta, 40000)[1]
        # The reason we only use the last 20000 values of the list is explained later
        tau, chi = correlation_time(m_list[20000:])
        tau_list = np.append(tau_list, tau)

plt.plot(T_list, tau_list)
plt.title(r"Correlation time, $\tau$, as a function of temperature")
plt.xlabel(r"$T$ [K]")
plt.ylabel(r"$\tau$")
plt.show()

Этот рисунок иллюстрирует важность использования различных времен корреляции для разных температур. Обратите внимание на пик времени корреляции около $T=2,25$ K. Как мы увидим позже, эта температура является критической температурой для фазового перехода системы. Этот эффект называется "критическим замедлением". Эффект критического замедления может быть уменьшен путем настройки алгоритма метрополиса. Более подробную информацию по этой теме можно найти в главе 4 в [[3]](#rsc). Определяя $\tau$ для каждого моделирования, мы можем извлечь из нашей системы как можно больше независимых измерений, тем самым минимизируя ошибки в наших результатах.

### Равновесное время

Последняя величина, которую нам нужно получить, прежде чем мы сможем запустить полное моделирование Монте-Карло, - это время равновесия $\tau_{\mathrm{eq}} $. $\tau_{\mathrm{eq}} $ - это мера того, сколько разверток Монте-Карло необходимо, прежде чем система достигнет равновесия. К счастью, метод определения $\tau_{\mathrm{eq}} $ намного проще, чем определение $\tau$. Мы можем просто запустить небольшое количество симуляций для одного и того же $T$ и посмотреть, когда указанное значение ($m$ или $E$) начнет стабилизироваться.

Однако $\tau_{\mathrm{eq}} $ масштабируется с $N$ и варьируется от моделирования к моделированию. Всегда следует переоценивать $\tau_{\mathrm{eq}}$, потому что наши измерения действительны только тогда, когда система находится в равновесии, и начало измерений до этого момента сделало бы наши результаты бесполезными. Особенно при низких температурах $\tau_{\mathrm{eq}} $ может быть огромным (до 150 000 шагов Монте-Карло). Это происходит, когда система падает до локального энергетического минимума и не может еще больше снизить свою энергию. Важно отметить, что $\tau_{\mathrm{eq}} $ обычно будет низким для низких температур, но достижение равновесия может занять много времени, поэтому нам всегда нужно переоценивать $\tau_{\mathrm{eq}}$, чтобы быть в безопасности.

Ниже мы построим график намагниченности в зависимости от развертки Монте-Карло, чтобы проиллюстрировать, как можно определить $\tau_{\mathrm{eq}} $.


In [ ]:
N = 128
S = 2 * np.random.randint(2, size=(N, N)) - 1

# High temperature
T = 5
beta = 1 / T
m_list = compute_quantities_fortran(S, beta, 30000)[1]
step_list = np.arange(30000)
plt.plot(step_list, m_list, label= "High temperature")


# Medium temperature
T = 2.2
beta = 1 / T
m_list = compute_quantities_fortran(S, beta, 30000)[1]
plt.plot(step_list, m_list, label= "Medium temperature")

# Low temperature
T = 1
beta = 1 / T
m_list = compute_quantities_fortran(S, beta, 30000)[1]
plt.plot(step_list, m_list, label= "Low temperature")


plt.xlabel(r"$t$")
plt.ylabel(r"$m$")
plt.title(r"The development of the magnetization as a function of $t$, at different temperatures")
plt.legend()
plt.show()

В случае высокой температуры мы видим, что система в основном находится в равновесии с самого начала моделирования, в то время как в случае средней температуры система достигает равновесия примерно через 8000 циклов. Мы также видим пример, когда системе требуется много времени, чтобы достичь равновесия. На самом деле, в случае низкой температуры система все еще не достигла равновесия после 30000 проходов. Это иллюстрирует важность использования высокого $\tau_{\mathrm{eq}} $. К счастью, в нашем распоряжении есть тонны вычислительной мощности, поэтому мы можем позволить себе иметь высокий $\tau_{\mathrm{eq}} $. Еще один важный момент заключается в том, что знак намагниченности не имеет никакого физического значения до тех пор, пока нет внешнего поля. Если намагниченность отрицательная, можно просто повернуть всю систему (перевернув ее вверх дном), чтобы получить положительную намагниченность. В дальнейшем мы будем говорить о намагниченности и $\textit{величине}$ намагниченности взаимозаменяемо.


## Full simulations

Давайте, наконец, объединим все наши вспомогательные функции и теорию в единую функцию. 

In [ ]:
def full_simulation(K, N, sweeps, T_min = 1, T_max = 4, B = 0, J_v = 1, J_p = 1, show_time = False):
    """Runs the full Monte Carlo simulation and returns the magnetization and specific heat as a function of
    temperature. The temperature interval is controlled by the user.
    
        Parameters:
            K: Number of temperatures we want simulated
            N: Number of columns and rows in the lattice
            sweeps: Number of sweeps we want to run after the system has reached equlibrium
            T_min: Minimum temperature of our simulation
            T_max: Maximum temperature of our simulation
            B, J_v, J_p: Interactation and external parameters
        Returns:
            m: The magnetization as a function of temperature, (Kx1) array
            c: The specific heat as a function of temperature, (Kx1) array
            T: List of the temperatures that was used in the simulation, (Kx1) array
        """
    T = np.linspace(T_min, T_max, K)
    betas = 1/T
    m = np.zeros(K)
    c = np.zeros(K)
    for i in range(K):
        S = 2 * np.random.randint(2, size=(N, N)) - 1
        # Seperating the two cases (High/Low tau_equil)
        if 1/betas[i] < 2.5:
            if N < 129: # Also seperating the biggest lattices from the smaller ones as they have a higher tau_eq
                tau_equil = 60000
                He_list, m_list = compute_quantities_fortran(S, betas[i], tau_equil + sweeps, B, J_v, J_p, show_time)
            else:
                tau_equil = 300000
                He_list, m_list = compute_quantities_fortran(S, betas[i], tau_equil + sweeps, B, J_v, J_p, show_time)
        else:
            if N < 129: # Also seperating the biggest lattices from the smaller ones as they have a higher tau_eq
                tau_equil = 10000
                He_list, m_list = compute_quantities_fortran(S, betas[i], tau_equil + sweeps, B, J_v, J_p, show_time)
            else:
                tau_equil = 50000
                He_list, m_list = compute_quantities_fortran(S, betas[i], tau_equil + sweeps, B, J_v, J_p, show_time)
            
        try: # If the system is too stable (i.e no notable changes during the simulations), the correlation_time
            # function will not work. That is why we use a try/expect
            tau_m = correlation_time(m_list[tau_equil:])[0] 
            step_size_m = int(2 * tau_m + 1)  # Independant magnetization values
            
            
            # Same for E and c, we assume that they have same equilibrium time as m
            tau_E = correlation_time(He_list[tau_equil:])[0]
            step_size_E = int(2 * tau_E + 1)
            
        # At extremly low temperatures, there is no need to find the correlation time, because the lattice almost 
        # does not change. In that case, we just use step_size_m = 10, step_size_E = 10
        except:
            step_size_m = 10
            step_size_E = 10

            
        # Calculating the average magnetization
        m[i] = np.sum(m_list[tau_equil::step_size_m]) / len(m_list[tau_equil::step_size_m])


        He_list_ind = He_list[tau_equil::step_size_E]
        # Calculating the average energy and the average squared energy
        He = np.sum(He_list_ind) / np.shape(He_list_ind)
        E_squared = np.sum(np.power(He_list_ind, 2)) / np.shape(He_list_ind)
        # Calculating the specific heat by determining the energy fluctuations (as mentioned in the theory)
        c[i] = betas[i] ** 2 * (E_squared - He ** 2)
    
    # Sign of magnetization is not of physical importance
    m = np.absolute(m)
    
    # Want the specific heat independant of the lattice size (i.e. the specific heat per lattice site)
    c = c / (N ** 2)
    return m, c, T



## Аналитическое решение Онсагера

Теперь, когда мы реализовали наше моделирование с достаточной вычислительной скоростью, мы можем, наконец, начать выполнять полное моделирование и сравнивать его с теоретическими результатами. Как упоминалось во введении, Ларс Онсагер нашел аналитическое решение для намагниченности и удельной теплоты. Для этого требуется решетка $N \times \infty$, но при использовании большого $N$ наше моделирование даст результаты, сопоставимые с результатами Ларса Онсагера. Решение Онсагера также требует периодических граничных условий, которые придают нашей решетке топологию тора и что магнитное поле $B = 0$. Производные следующих уравнений выводятся в [[1]](#rsc), [[2]](#rsc) и [[3]](#rsc), но для простоты результаты будут изложены без доказательств. 

Намагниченность принимает форму 

\begin{equation}
m = \left\{
  \begin{array}{lr}
    0 & : T > T_c\\
    \big\{1- \big[ \mathrm{sinh}(2 \beta J) \big]^{-4} \big\}^\frac{1}{8} & : T < T_c
  \end{array}
\right.
\label{magnetization}
\end{equation}

Мы предположили здесь, что $J=J'$. Это спонтанное намагничивание, которое возникает без индуцирования внешним полем.
Выражение для удельной теплоты является более сложным и может быть выражено более элегантно путем введения

\begin{equation}
\kappa \equiv \frac{2 \mathrm{sinh}(2 \beta J)}{\mathrm{cosh}^2(2 \beta J) }, \quad \kappa ' \equiv 2 \mathrm{tanh}^2(2 \beta J) - 1. 
\label{kappas}
\end{equation}

Нам также нужны выражения для полных эллиптических интегралов первого и второго рода, обозначаемых как $K_1(\kappa)$ и $E_1(\kappa)$ соответственно. Они представляют собой табличные функции, заданные

\begin{equation}
 K_1(\kappa) \equiv \int_0^{\frac{\pi}{2}} \frac{d\phi}{\sqrt{1- \kappa^2 \sin^2{\phi}}}, \quad E_1(\kappa) \equiv \int_0^{\frac{\pi}{2}} d\phi\sqrt{1- \kappa^2 \sin^2{\phi}}.
\label{eliptic_integrals}
\end{equation}

Теперь мы можем, наконец, выразить удельную теплоту как 

\begin{equation}
\frac{C_B}{N² k_B} = \frac{2}{\pi} (\beta J \mathrm{coth}(2 \beta J) ^2 \Big\{ 2 K_1(\kappa) - 2 E_1(\kappa) - (1-\kappa') \big[\frac{\pi}{2} + \kappa' K_1(\kappa) \big] \Big\}.
\label{C_B} \quad(4)
\end{equation}

Это уравнение имеет особенность при $\kappa' = 0$, что соответствует критической температуре $T_c$. Решение $\kappa' = 0$ для T дает следующую критическую температуру
\begin{equation}
T_c = \frac{2J}{\ln({1 + \sqrt{2})}} \simeq 2.269 J. 
\label{T_c}
\end{equation}

Все термодинамические функции имеют сингулярность в той или иной форме при $T_c$. Такое поведение указывает на то, что у нас есть фазовый переход. Выше $T_c$ система находится в парамагнитной фазе, где средняя намагниченность равна нулю. Ниже $T_c$ система находится в ферромагнитной фазе, где она развивает спонтанное намагничивание. Для $T$ вблизи $T_c$ удельная теплота приближается к бесконечности логарифмически, как $|T-T_c| \rightarrow 0$.



Теперь помните, что решение Онсагера справедливо только для бесконечной решетки, поэтому мы ожидаем, что более крупные решетки будут лучше всего аппроксимировать его решения. Давайте начнем с использования малой, средней и большой решетки.

In [ ]:
# Small lattice
N = 8
m_s, c_s, T = full_simulation(20, N, 200000)

# Medium lattice
N = 32
m_m, c_m, T = full_simulation(20, N, 200000)

# Large lattice
N = 128
m_l, c_l, T = full_simulation(20, N, 200000)

# The T is equal for all the simulations

# Magnetization
plt.scatter(T, m_s, label=r"$N$ = 8", marker='*')
plt.scatter(T, m_m, label=r"$N$ = 32", marker='o')
plt.scatter(T, m_l, label=r"$N$ = 128", marker='x')

T_2 = np.linspace(1, 5, 10000)
m_analytical = (1 - (np.sinh(2 / T_2)) ** (-4)) ** (1 / 8) # Onsager's analytical solution
plt.plot(T_2, m_analytical, label = "Analytic solution")
plt.axvline(2.269, ymax=0.4) # Adding vertical line to illustrate that the functions comes screaming down at T_c
plt.legend()
plt.title(r"Comparison of the numerical and analytical results for the magnetization")
plt.xlabel(r"$T$ [K]")
plt.ylabel(r"$m$")
plt.show()


# Specific heat
plt.scatter(T, c_s, label=r"$N$ = 8", marker='*')
plt.scatter(T, c_m, label=r"$N$ = 32", marker='o')
plt.scatter(T, c_l, label=r"$N$ = 128", marker='x')

plt.xlabel(r"$T$ [K]")
plt.ylabel(r"$c$")
plt.title(r"Comparison of the numerical and analytical results for the specific heat")
beta = 1 / T_2
# Assuming J_v = J_p = 1
J_v = 1
J_p = 1
kappa_1 = 2 * np.sinh(2 * beta * J_v) / (np.cosh(2 * J_v * beta) ** 2 )
kappa_2 = 2 * np.tanh(2 * beta * J_v) ** 2 - 1

elliptical_first_kind = scipy.special.ellipk(kappa_1)
elliptical_second_kind = scipy.special.ellipe(kappa_1)

C_v_analytical = 2 / np.pi * (beta * J_v) ** 2 / np.tanh(2 * beta * J_v) ** 2 *(
    2 * elliptical_first_kind - 2 * elliptical_second_kind -
     (1 - kappa_2) * (np.pi / 2 + kappa_2 * elliptical_first_kind))
plt.plot(T_2, C_v_analytical, label= "Analytic solution")
plt.legend()
plt.show()



Из первого рисунка видно, что две самые большие решетки с $N=32$ и $N=128$ соответственно довольно хорошо воспроизводят аналитические решения для намагниченности на бесконечной решетке. Во всех трех симуляциях при температурах выше $T_c$ намагниченность равна нулю, т. е. спонтанной намагниченности нет. Однако мы наблюдаем превосходство самой большой решетки при измерении ближе к $T_c$. Здесь производная намагниченности бесконечна, поэтому, хотя кажется, что две меньшие решетки почти имеют правильное значение для $m$, отклонение на самом деле довольно велико, так как намагниченность падает до $T_c$. Пренебрегая измерением при $T_c$, мы видим, что можем использовать $N=32$ и примерно получить те же результаты, что и для более крупной решетки. Моделирование с $N=32$ выполняется в восемь раз быстрее, чем $N=128$, что является существенной разницей, поэтому мы приходим к выводу, что для намагничивания мы можем уйти, используя только $N=32$ за пределами критической области. Но для моделирования ближе к $T_c$ мы отмечаем, что чем больше решетка, тем лучше результат.

Что касается теплоемкости, то мы наблюдаем аналогичную тенденцию. За пределами критической области все три решетки на самом деле преформируются одинаково хорошо, так как все они немного недооценивают теплоемкость. Поскольку $c$ логарифмически расходится при $T_c$, мы также наблюдаем резкое увеличение удельной теплоты при этой температуре для $N=32$ и $N=128$, причем большая решетка увеличивается больше всего.

Основываясь на наших результатах, может показаться, что использование большого $N$ не нужно, так как результаты моделирования с $N=32$ почти идентичны моделированию с $N=128$. Это верно, но наш главный интерес заключается в фазовом переходе, который происходит при $T_c$. Таким образом, нам действительно нужно использовать большой $N$, чтобы изучить поведение системы, близкое к $T_c$, потому что $N$ существенно влияет на термодинамические величины в этой области. Мы рассмотрим критическую область более подробно позже.

Ларс Онсагер действительно нашел формулу для удельной теплоты для общих $J$ и $J'$, но формула становится намного сложнее, чем уравнение (4). Поэтому мы позаимствуем одну из его цифр из его статьи, чтобы сравнить ее с нашим численным результатом. В этом случае мы используем $ J = 1$, в то время как $ J'$ будет принимать значения 1, 0,01 и 0. Здесь мы используем довольно большое $N$, потому что $J' = 0,01$ должен быть смоделирован с большой решеткой, чтобы получить результаты, сопоставимые с результатом Онсагера. 

In [ ]:
# These simulations were run overnight, and are split up 
N = 256

J_p = 1
m_1, c_1, T_1 = full_simulation(30, N, 200000, T_min = 0.3, T_max = 3, J_p = J_p)

In [ ]:
N = 256

J_p = 0.01
m_2, c_2, T_2 = full_simulation(30, N, 200000, T_min = 0.3, T_max = 1.5, J_p = J_p)

In [ ]:
N = 256

J_p = 0
m_3, c_3, T_3 = full_simulation(30, N, 200000, T_min = 0.3, T_max = 1.5, J_p = J_p)

In [ ]:
J_p = 1
plt.plot(T_1, c_1, label= "$J'$ = {}".format(J_p), linestyle = '-.')

J_p = 0.01
# Multiply the temperature with 2 to have it on the same format as Onsager 
plt.plot((T_2 * 2), c_2, label= "$J'$ = {}".format(J_p), linestyle = '-')

J_p = 0
# Multiply the temperature with 2.02 to have it on the same format as Onsager
plt.plot((T_3 * 2.02), c_3, label= "$J'$ = {}".format(J_p), linestyle = '--')


plt.xlabel(r"$\frac{2}{H-H'}$ [K]")
plt.ylabel(r"$c$")
plt.title(r"Specific heat, $c$, with different values for $J'$. $N$ = {}".format(N))
plt.legend()
plt.show()

![Onsager_figure.png](images\Onsager_figure.png)

Ларс Онсагер использовал несколько иную нотацию, чем мы, когда он построил график теплоемкости против $\frac{2}{H-H'}$, где $H = \beta J$ и $H' = \beta J'$. В случае анизотропного моделирования ($J \neq J'$) мы можем преобразовать температуру в его единицу измерения, просто умножив температуру на $\frac{2}{J-J'}$. При $J'=0$ мы наблюдаем, что заново открываем знаменитое решение Эрнста Изинга для теплоемкости одномерной цепи Изинга с периодическими граничными условиями (кольцо), поскольку у нас просто есть $N$ таких колец, которые не взаимодействуют. В изотропном случае мы снова обнаруживаем, что наше моделирование дает результаты, сопоставимые с (4), и поскольку мы использовали большую решетку, чем ранее, мы получаем еще лучшие результаты.

Для $J'=0,01 J$ должен быть фазовый переход при $\frac{2}{H-H'} = 1$, где температура равна $T_c'$, а теплоемкость расходится. Однако, как видно из рисунка Онсагера, это расхождение влияет только на температуры, очень близкие к критической температуре, поэтому, если мы не запустим наше моделирование при температуре, очень близкой к критической температуре, мы не увидим ничего похожего на расхождение. Качественно наши результаты для $J'=0,01 J$ очень хорошо согласуются с цифрой Онсагера. Мы видим, что теплоемкость начинает быть меньше, чем в случае $J'=0$, и непосредственно перед тем, как температура достигнет $T_c'$, происходит резкое увеличение с соответствующим быстрым уменьшением после $T_c'$. Это указывает на то, что в нашей системе при $T_c'$ происходит что-то радикальное, то есть указывает на фазовый переход. Для более высоких температур мы наблюдаем, что теплоемкость сходится к случаю $J'=0$, поскольку связь между взаимодействиями между столбцами слишком слаба, чтобы повлиять на систему. 

Теперь давайте для полноты картины проведем моделирование, имеющее внешнее магнитное поле, и посмотрим, как реагирует решетка. Помните, что, поскольку $B \neq 0$, решение Онсагера больше недействительно, как упоминалось ранее, для этой проблемы нет известного аналитического решения. Тем не менее, мы все еще можем попытаться понять результаты с физической точки зрения и обосновать их обоснованность, основываясь на простых физических принципах.

In [ ]:
N = 128
m, c, T = full_simulation(45, N, 300000, B = 1, T_max = 10)

plt.plot(T, m)
plt.xlabel(r"$T$ [K]")
plt.ylabel(r"$m$")
plt.title(r"Magnetization, $m$, with nonzero exernal magnetic field, $B=1$ and $N$ = {}".format(N))
plt.show()

plt.plot(T, c)
plt.xlabel(r"$T$ [K]")
plt.ylabel(r"$c$")
plt.title(r"Specific heat, $c$, with nonzero exernal magnetic field, $B=1$ and $N$ = {}".format(N))
plt.show()


Здесь мы видим, что у нас ненулевая намагниченность при гораздо более высоких температурах, чем раньше. Объяснение этому простое: чтобы минимизировать энергию системы, спины стараются указывать в том же направлении, что и $B$. Однако внешнее магнитное поле все еще слишком слабо, чтобы выровнять все спины при температурах выше 2 К. Как тепловые возбуждения системы, так и параметры взаимодействия $J$ и $J'$ работают против полного выравнивания спина.

Удельная теплоемкость также иллюстрирует эту борьбу за власть между $B$ и его противниками. Чем более равномерно они будут согласованы, тем больше будет спиновых флуктуаций, и, следовательно, флуктуации энергии также увеличатся. Как мы видели ранее, удельная теплоемкость пропорциональна колебаниям энергии, поэтому, когда $c$ имеет максимум, это означает, что борьба за выравнивание наиболее интенсивна. При 4 К мы наблюдаем, что $c$ имеет максимум, поэтому мы приходим к выводу, что война между $B$ и $J$, $J'$ и тепловыми возбуждениями бушует в худшем случае при этой температуре.

### Критические показатели

Поскольку мы посвятили так много времени получению большой вычислительной мощности, мы хотим использовать нашу модель, чтобы найти один из критических показателей двумерной модели Изинга. Критические показатели описывают поведение термодинамических величин вблизи фазовых переходов. В более высоких измерениях, поскольку нет известных аналитических решений, была проделана большая численная работа по определению точных оценок критических показателей. Критические показатели чрезвычайно чувствительны к отклонениям от аналитического результата, поэтому теперь мы хотим довести нашу модель до ее пределов.

Критический показатель степени $\beta$ (не путать с обратной температурой, для обозначения которой мы до сих пор использовали $\beta$) описывает, как намагниченность исчезает вблизи $T_c$. В этой температурной области намагниченность может быть записана как

$$ 
m \sim \epsilon |T-T_C|^\beta,
$$
где $\epsilon$ - некоторая константа. Раскладывая аналитическое решение уравнения намагниченности в ряд Тейлора по $m$, можно обнаружить, что 2D-модель бесконечного Изинга имеет $\beta = \frac{1}{8}$. Давайте посмотрим, как работает наша модель. 

In [ ]:
def mag_low_T(T, epsilon, beta):
    """Dummy function declariation that is needed to use the curve_fit function from the scipy library. 
    
        Parameters:
            T: Temperature
            epsilon: Fitting parameter
            beta: Fitting parameter, the critical exponent for magnetization with no external field
        Returns:
            m: epsilon * T ** beta
        """
    m = epsilon * T ** beta
    return m

def critical_exponent(K, N, sweeps, T_low, show_time = False):
    """Function that calculates the critical exponent of the 2D Ising model with B=0, and J_v = J_p = 1. Runs a Monte
    Carlo simulation with temperatures close to T_c and estimates the critical exponent, uses the simualtions with T
    between T_low and T_c
    
        Parameters:
            K: Number of temperatures we want simulated
            N: Number of columns and rows in the lattice
            T_low: The start of the temperature interval the simulations are ran with.
        Returns:
            m_list: The magnetization as a function of deviation from the critical temperature, (Kx1) array
            c_list: The specific heat as a function of deviation from the critical temperature, (Kx1) array
            T_list: Contains the temperatures the simulations are run with, more accurately how much they
                    deviate from T_c, (Kx1) array
        """
    T_c = 2.26918 # Can be derived from Onsager's equaitons
        
    T_list = np.linspace(T_low, T_c, K)
    m_list = full_simulation(K, N, sweeps, T_low, T_c)[0]
    deviation_from_T_c = np.absolute(T_list - T_c)
    params = scipy.optimize.curve_fit(mag_low_T, deviation_from_T_c, m_list)[0]
    beta = params[1]
    epsilon = params[0]
    print("N = ", N, ", \t", r"beta = ", "%.4f" % beta)
    return m_list, beta, deviation_from_T_c, epsilon
    



In [ ]:
# Again we split the simulations
m_64, beta_64, T_list_64, epsilon_64 = critical_exponent(25, 64, 200000, 2)


In [ ]:
m_128, beta_128, T_list_128, epsilon_128 = critical_exponent(25, 128, 600000, 2)

In [ ]:
m_256, beta_256, T_list_256, epsilon_256 = critical_exponent(25, 256, 300000, 2)

Как и ожидалось, критические показатели из нашего моделирования постепенно приближаются к $\beta = \frac{1}{8}=0.125$, по мере увеличения $N$. Самая маленькая решетка резко переоценивает $\beta$, поэтому эти моделирования действительно иллюстрируют важность использования большой решетки при сравнении с результатами бесконечной решетки. Прелесть определения критических показателей заключается в том, что они не зависят от экспериментальных параметров, таких как $J$, они обычно зависят только от размерности системы. Поскольку $J$ практически невозможно надежно измерить, критические показатели гораздо важнее при описании фазового перехода.


Создание собственной модели Изинга - отличное упражнение, особенно если оно заставляет вас поддерживать свой код чистым и эффективным. С современными компьютерами легко быть небрежным при написании кода, так как он, скорее всего, будет работать более чем достаточно быстро. Наш последний вызов функции запустил самые основные части кода триллион раз! Владение высокоскоростным языком, таким как Фортран, является отличным инструментом для вычислительного физика. Следует отметить, что некоторые из наиболее трудоемких симуляций (особенно с $N>128$) в этом ноутбуке выполнялись ночью, поэтому, хотя у нас было много вычислительной мощности, нам все еще требовалось время для получения надежных результатов. Мы настоятельно рекомендуем книгу Ньюмана и Баркемы [[4]](#rsc), если вы хотите изучить более сложные методы Монте-Карло, чем алгоритм Метрополиса, и продолжить изучение различных термодинамических величин модели Изинга, таких как, например, магнитная восприимчивость, $\chi$ или если вы хотите расширить модель Изинга до трех измерений. С моделью Ising еще многое предстоит узнать и сделать!

## Благодарности
Особая благодарность проф. Асле Судбе за создание проекта, предоставление глубоких комментариев и за заимствование необходимой литературы. 

___
<a id="rsc"></a>
## Resources and further reading
<a>[1]</a>: L. Onsager, *Crystal Statistics. I. A Two-Dimensional Model with an Order-Disorder Transition*, American Physical Society, 1944. <br />
<a>[2]</a>: K. Huang, *Statistical Mechanics*, John Wiley & Sons, 1987. <br />
<a>[3]</a>: Yang, C. N, *The Spontaneous Magnetization of a Two-Dimensional Ising Model*, American Physical Society, 1952. <br />
<a>[4]</a>: Newman, M. E. J & Barkema, G. T, *Monte Carlo methods in Statistical Physics*, Oxford University Press, 1999. <br />

## Appendix
We give the Fortran(90) subroutine below, which was converted into a Python Module with F2PY.

``` fortran
! Author: Eilif Sommer Øyre and Niels Henrik Aase
! Runs a single Monte Carlo sweep on a 2D Ising lattice
! The boundary conditions are periodic.


subroutine random_sweep(del_He_sweep, del_m_sweep, S, S_initial, beta, &
  B, J_v, J_p, rand_numbers, rand_list_index1, rand_list_index2, N, N_squared)
  
  implicit none
  ! Declearing arguments
  integer ,intent(in)  :: N, N_squared
  real    ,intent(in)  :: beta, B, J_v, J_p
  real    ,intent(in)  :: rand_numbers(N_squared)
  integer ,intent(in)  :: rand_list_index1(N_squared), rand_list_index2(N_squared)
  integer ,intent(in)  :: S_initial(N,N)
  ! Declearing private parameters
  integer ,intent(out) :: S(N,N)
  real    ,intent(out) :: del_He_sweep, del_m_sweep
  real                 :: nn, del_E, del_m
  logical              :: check
  integer              :: j, k, l
  integer              :: k_neighbour_under, k_neighbour_over
  integer              :: l_neighbour_right, l_neighbour_left

  del_He_sweep = 0
  del_m_sweep = 0

  S = S_initial

  do j = 1,N_squared
     l = rand_list_index2(j)
     k = rand_list_index1(j)

     ! Because of periodic boundary conditions nearest
     ! neighbours indecies must be carefully examined.
     ! If position (k, l) is on the bottom row:
     if (k==N) then
        k_neighbour_over = 1
        k_neighbour_under = k - 1
        ! If position (k, l) is on the top row:
     elseif (k==1) then
        k_neighbour_over = k + 1
        k_neighbour_under = N
     else
        k_neighbour_over = k + 1
        k_neighbour_under = k - 1
     endif

     ! If position (k, l) is on the righmost column:
     if (l==N) then
        l_neighbour_right = 1
        l_neighbour_left = l - 1
     ! If position (k, l) is on the leftmost column:
     elseif (l==1) then
        l_neighbour_right = l + 1
        l_neighbour_left = N
     else
        l_neighbour_right = l + 1
        l_neighbour_left = l - 1
     endif

     ! Calculating energy contribution from nearest neighbours
     nn = J_v*(S(k_neighbour_under, l) + S(k_neighbour_over, l)) + J_p* &
                      ( S(k, l_neighbour_right) + S(k, l_neighbour_left))
     del_E = 2*S(k,l)*(nn + B)

     if (del_E <= 0) then
        check = .true.
     else
        check = (exp(-beta*del_E) > rand_numbers(j))
     endif

     if (check) then
        del_m = -2*S(k,l)/ real(N**2)
        S(k,l) = -1*S(k,l)

     else
        del_m = 0
        del_E = 0
     endif

     del_He_sweep = del_He_sweep + del_E
     del_m_sweep = del_m_sweep + del_m
  enddo
  return

end subroutine random_sweep
```